# What is Tensorflow?

TensorFlow is a Python library that allows users to express arbitrary computation as a graph of data flows. Nodes in this graph represent mathematical operations, whereas edges represent data that is communicated from one node to another. Data in TensorFlow is represented as tensors, which are multidimensional arrays (representing vectors with a 1D tensor, matrices with a 2D tensor, etc.).

## Installing Tensorflow

$ pip install --upgrade tensorflow      # for Python 2.7 
$ pip3 install --upgrade tensorflow     # for Python 3.n
$ pip install --upgrade tensorflow-gpu  # for Python 2.7  and GPU 
$ pip3 install --upgrade tensorflow-gpu # for Python 3.n  and GPU 

* Note: Do not forget to install CUDA for tensorflow GPU if you have a Nvidia GPU

In [1]:
import tensorflow as tf

dl=tf.constant('Testing tensorflow')
session=tf.Session()
session.run(dl)

b'Testing tensorflow'

In [2]:
a=tf.constant(2)
b=tf.constant(3)
multiply=tf.multiply(a,b)
session.run(multiply)

6

In [3]:
add=tf.add(a,b) #add
session.run(add)

5

## Creating and Manipulating Tensorflow Variables

When we build a deep learning model in TensorFlow, we use variables to represent the parameters of the model. TensorFlow variables are in-memory buffers that contain tensors; but unlike normal tensors that are only instantiated when a graph is run and that are immediately wiped clean afterward, variables survive across multiple executions of a graph. As a result, TensorFlow variables have the following three properties:

* Variables must be explicitly initialized before a graph is used for the first time.

* We can use gradient methods to modify variables after each iteration as we search for a model’s optimal parameter settings.

* We can save the values stored in variables to disk and restore them for later use. 

These three properties are what make TensorFlow especially useful for building machine learning models. 

Creating a variable is simple, and TensorFlow provides mechanics that allow us to initialize variables in several ways. Let’s start off by initializing a variable that describes the weights connecting neurons between two layers of a feed-forward neural network:

In [4]:
weigths=tf.Variable(tf.random_normal([300,200],stddev=0.5),name='weights')

Here we pass two arguments  to tf.Variable.6 The first, tf.random_normal,7 is  an operation that produces a tensor initialized using a normal distribution with standard deviation 0.5. We’ve specified that this tensor is of size 300 x 200, implying that the weights connect a layer with 300 neurons to a layer with 200 neurons. We’ve also passed a name to our call to tf.Variable.  The name is a unique identifier that allows us to refer to the appropriate node in the computation graph. In this case, weights is meant to be trainable; or in other words, we will automatically compute and apply gradients to weights. If weights is not meant to be trainable, we may pass an optional flag when we call tf.Variable:


In [5]:
weigths=tf.Variable(tf.random_normal([300,200],stddev=0.5),name='weights',trainable=False)

In addition to using tf.random_normal, there are several other methods to initialize a TensorFlow variable:


In [6]:
# common tensors from the Tensorflow API docs
tf.zeros([100,50],dtype=tf.float32,name=None)

<tf.Tensor 'zeros:0' shape=(100, 50) dtype=float32>

In [7]:
tf.ones([100,50],dtype=tf.float32,name=None)

<tf.Tensor 'ones:0' shape=(100, 50) dtype=float32>

In [8]:
tf.random_normal([100,50],mean=0,stddev=1.0,dtype=tf.float32,seed=None)

<tf.Tensor 'random_normal_2:0' shape=(100, 50) dtype=float32>

In [9]:
tf.truncated_normal([100,50],mean=0,stddev=1.0,dtype=tf.float32,seed=None,name=None)

<tf.Tensor 'truncated_normal:0' shape=(100, 50) dtype=float32>

In [10]:
tf.random_uniform([100,50],minval=0,maxval=None,dtype=tf.float32,seed=None,name=None)

<tf.Tensor 'random_uniform:0' shape=(100, 50) dtype=float32>

When we call tf.Variable, three operations are added to the computation graph: 

* The operation producing the tensor we use to initialize our variable.
* The tf.assign operation, which is responsible for filling the variable with the initializing tensor prior to the variable’s use.
* The variable operation, which holds the current value of the variable.

This can be visualized as shown
<img src='images/image1.PNG'>

As we mentioned previously in the three operations, before we use any TensorFlow variable, the tf.assign operation must be run so that the variable is appropriately initialized with the desired value. We can do this by running tf.initial ize_all_variables(), which will trigger all of the tf.assign operations in our graph. We can also selectively initialize only certain variables in our computational graph using the tf.initialize_variables(var1, var2, ...). We’ll describe this in more detail when we discuss sessions in TensorFlow.


## Tensorflow Operations

We’ve already talked a little bit about operations in the context of variable initialization, but these only make up a small subset of the universe of operations available in TensorFlow. On a high level, TensorFlow operations represent abstract transformations that are applied to tensors in the computation graph. Operations may have attributes that may be supplied a priori or are inferred at runtime. For example, an attribute may serve to describe the expected types of input (adding tensors of type float32 versus int32). Just as variables are named, operations may also be supplied with an optional name attribute for easy reference into the computation graph.

An operation consists of one or more kernels, which represent device-specific implementations. For example, an operation may have separate CPU and GPU kernels because it can be more efficiently expressed on a GPU. This is the case for many TensorFlow operations on matrices.

<img src='images/image2.PNG'>


## Placeholder Tensors

Now that we have a solid understanding of TensorFlow variables and operations, we have a nearly complete description of the components of a TensorFlow computation graph. The only missing piece is how we pass the input to our deep model (during both train and test time). A variable is insufficient because it is only meant to be initialized once. Instead, we need a component that we populate every single time the computation graph is run.

TensorFlow solves this problem using a construct called a placeholder. A placeholder is instantiated as follows and can be used in operations just like ordinary TensorFlow variables and tensors:

In [11]:
x=tf.placeholder(tf.float32,name='x',shape=[None,784])
W=tf.Variable(tf.random_uniform([784,10],-1,1),name='W')
multiply=tf.matmul(x,W)

Here we  define a placeholder where x represents a minibatch of data stored as float32’s. We notice that x has 784 columns, which means that each data sample has 784 dimensions. We also notice that x has an undefined number of rows. This means that x can be initialized with an arbitrary number of data samples. While we could instead multiply each data sample separately by W, expressing a full minibatch as a tensor allows us to compute the results for all the data samples in parallel. The result is that the ith row of the multiply tensor corresponds to W multiplied with the ith data sample. 

Just as variables need to be initialized the first time the computation graph is built, placeholders need to be filled every time the computation graph (or a subgraph) is run. We’ll discuss how this works in more detail in the next section.

## Sessions in Tensorflow

A TensorFlow program interacts with a computation graph using a session.13 The TensorFlow session is responsible for building the initial graph, and can be used to initialize all variables appropriately and to run the computational graph. To explore each of these pieces, let’s consider the following simple Python script:


In [12]:
x=tf.placeholder(tf.float32,name='x',shape=[None,784])
W=tf.Variable(tf.random_uniform([784,10],-1,1),name='W')
b=tf.Variable(tf.zeros([10]),name='biases')
output=tf.matmul(x,W)+b

init_op=tf.initialize_all_variables()

sess=tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


The first four lines after the import statement describe the computational graph that is built by the session when it is finally instantiated. The graph (sans variable initialization operations) is depicted in Figure. We then initialize the variables as required by using the session variable to run the initialization operation in sess.run(init_op). Finally, we can run the subgraph by calling sess.run again, but this time we pass in the tensors (or list of tensors) we want to compute.

<img src='images/image3.PNG'>

Finally, the sess.run interface can also be used to train networks. We will explore this in further detail when we use TensorFlow to train our first machine learning model on MNIST. But how exactly does a single line of code (sess.run) accomplish such a wide variety of functions? The answer lies in the powerful expressivity of the underlying computational graph. 

All of these functionalities are represented as TensorFlow operations that can be passed as arguments to sess.run. All sess.run needs to do is traverse down the computational graph to identify all of the dependencies that compose the relevant subgraph, ensure that all of the placeholder variables that belong to the identified subgraph are filled using the feed_dict, and then traverse back up the subgraph (executing all of the intermediate operations) to evaluate the original arguments. Now that we have a comprehensive understanding of sessions and how to run them, we’ll explore two more major concepts in building and maintaining computational graphs.

## Navigating Variable Scope and Sharing Variables

Although we won’t run into this problem just yet, building complex models often requires reusing and sharing large sets of variables that we’ll want to instantiate together in one place. Unfortunately, trying to enforce modularity and readability can result in unintended results if we aren’t careful. Let’s consider the following example:

In [13]:
def my_network(input):
    W_1=tf.Variable(tf.random_uniform([784,100],-1,1),name='W_1')
    b_1=tf.Variable(tf.zeros([100]),name='biases_1')
    output_1=tf.matmul(input,W_1)+b_1
    
    W_2=tf.Variable(tf.random_uniform([100,50],-1,-1),name='W_2')
    b_2=tf.Variable(tf.zeros([50]),name='biases_2')
    output_2=tf.matmul(output_1,W_2)+b_2
    
    W_3=tf.Variable(tf.random_uniform([50,10],-1,1),name='W_3')
    b_3=tf.Variable(tf.zeros([10]),name='biases_3')
    output_3=tf.matmul(output_2,W_3)+b_3
    
    # printing names
    print("Printing names of weight parameters")
    print(W_1.name,W_2.name,W_3.name)
    print("Printing names of biases parameters")
    print(b_1.name,b_2.name,b_3.name)
    
    return output_3

In [14]:
i_1=tf.placeholder(tf.float32,[1000,784],name='i_1')
my_network(i_1)

Printing names of weight parameters
W_1_1:0 W_2:0 W_3:0
Printing names of biases parameters
biases_1:0 biases_2:0 biases_3:0


<tf.Tensor 'add_4:0' shape=(1000, 10) dtype=float32>

In [15]:
i_2=tf.placeholder(tf.float32,[1000,784],name='i_2')
my_network(i_2)

Printing names of weight parameters
W_1_2:0 W_2_1:0 W_3_1:0
Printing names of biases parameters
biases_1_1:0 biases_2_1:0 biases_3_1:0


<tf.Tensor 'add_7:0' shape=(1000, 10) dtype=float32>

If we observe closely, our second call to my_network doesn’t use the same variables as the first call (in fact, the names are different!). Instead, we’ve created a second set of variables! In many cases, we don’t want to create a copy, but rather reuse the model and its variables. It turns out, that in this case, we shouldn’t be using tf.Variable. Instead, we should be using a more advanced naming scheme that takes advantage of TensorFlow’s variable scoping.

TensorFlow’s variable scoping mechanisms are largely controlled by two functions:

* tf.get_variable(name, shape, initializer) Checks if a variable with this name exists, retrieves the variable if it does, or creates it using the shape and initializer if it doesn’t.

    
* tf.variable_scope(scope_name) Manages the namespace and determines the scope in which tf.get_variable operates.

Let’s try to rewrite my_network in a cleaner fashion using TensorFlow variable scoping. The new names of our variables are namespaced as "layer1/W", "layer2/b", "layer2/W", and so forth:


In [16]:
def layer(input,weight_shape,bias_shape):
    weight_init=tf.random_uniform_initializer(minval=1,maxval=1)
    bias_init=tf.constant_initializer(value=0)
    W=tf.get_variable('W',weight_shape,initializer=weight_init)
    b=tf.get_variable('b',bias_shape,initializer=bias_init)
    return tf.matmul(input,W)+b

In [17]:
def my_network(input):
    with tf.variable_scope('layer_1'):
        output_1=layer(input,[784,100],[100])
    with tf.variable_scope('layer_2'):
        output_2=layer(output_1,[100,50],[50])
    with tf.variable_scope('layer_3'):
        output_3=layer(output_2,[50,10],[10])
    return output_3

In [18]:
# Now let’s try to call my_network twice, just like we did in the preceding code block:

i_1=tf.placeholder(tf.float32,[100,784],name='i_1')
my_network(i_1)

<tf.Tensor 'layer_3/add:0' shape=(100, 10) dtype=float32>

In [19]:
i_2=tf.placeholder(tf.float32,[1000,784],name='i_2')
my_network(i_2)

ValueError: Variable layer_1/W already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "c:\users\deepblue\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 1740, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "c:\users\deepblue\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\ops.py", line 3414, in create_op
    op_def=op_def)
  File "c:\users\deepblue\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


Unlike tf.Variable, the tf.get_variable command checks that a variable of the given name hasn’t already been instantiated. By default, sharing is not allowed (just to be safe!), but if we want to enable sharing within a variable scope, we can say so explicitly:


In [23]:
with tf.variable_scope("shared_variables") as scope:
    i_1=tf.placeholder(tf.float32,[1000,784],name="i_1")
    my_network(i_1)
    scope.reuse_variables()
    i_2= tf.placeholder(tf.float32,[1000,784],name="i_2")
    my_network(i_2)

This allows us to retain modularity while still allowing variable sharing. And as a nice byproduct, our naming scheme is cleaner as well.